In [1]:
import pandas as pd
import numpy as np

In [2]:
hourly_df = pd.read_csv("CryptoHourlyData.csv")
coins_df = pd.read_csv("coins.csv")

In [3]:
hourly_df.head()

,ID,Date,Time,Open,High,Low,Close
0,1,2023-09-10,00:00:00,56.27,56.29,56.16,56.18
1,1,2023-09-09,23:00:00,56.27,56.28,56.16,56.26
2,1,2023-09-09,22:00:00,56.14,56.28,56.12,56.26
3,1,2023-09-09,21:00:00,56.38,56.38,56.07,56.13
4,1,2023-09-09,20:00:00,56.20,56.33,56.19,56.31


In [4]:
coins_df.head()

,id,Symbol,name
0,1,AAVE,Aave
1,2,BTC,Bitcoin
2,3,ADA,Cardano
3,4,LINK,Chainlink
4,5,DOGE,Dogecoin


In [5]:
# Standardize column names
hourly_df.columns = hourly_df.columns.str.strip().str.lower()
coins_df.columns = coins_df.columns.str.strip().str.lower()

In [6]:
# Drop duplicates
hourly_df.drop_duplicates(inplace=True)
coins_df.drop_duplicates(inplace=True)

In [7]:
# Check for and drop missing values
hourly_df.dropna(subset=['id', 'date', 'time', 'open', 'high', 'low', 'close'], inplace=True)
coins_df.dropna(subset=['id', 'symbol'], inplace=True)

In [8]:
# Convert numeric columns to proper types
price_cols = ['open', 'high', 'low', 'close']
for col in price_cols:
    hourly_df[col] = pd.to_numeric(hourly_df[col], errors='coerce')

In [9]:
# Merge hourly data with coin metadata
merged = pd.merge(hourly_df, coins_df, how='left', left_on='id', right_on='id')

In [10]:
merged.head()

,id,date,time,open,high,low,close,symbol,name
0,1,2023-09-10,00:00:00,56.27,56.29,56.16,56.18,AAVE,Aave
1,1,2023-09-09,23:00:00,56.27,56.28,56.16,56.26,AAVE,Aave
2,1,2023-09-09,22:00:00,56.14,56.28,56.12,56.26,AAVE,Aave
3,1,2023-09-09,21:00:00,56.38,56.38,56.07,56.13,AAVE,Aave
4,1,2023-09-09,20:00:00,56.20,56.33,56.19,56.31,AAVE,Aave


In [11]:
# Combine 'date' and 'time' into a single datetime column
merged['datetime'] = pd.to_datetime(merged['date'] + ' ' + merged['time'], errors='coerce')
merged.dropna(subset=['datetime'], inplace=True)

In [12]:
# Filter out future dates or invalid timestamps
merged = merged[merged['datetime'] <= pd.Timestamp.now()]

In [13]:
# Sort by symbol and datetime
merged.sort_values(by=['symbol', 'datetime'], inplace=True)

In [14]:
# Remove extreme outliers
for col in price_cols:
    lower = merged[col].quantile(0.01)
    upper = merged[col].quantile(0.99)
    merged[col] = np.clip(merged[col], lower, upper)

In [15]:
merged.head()

,id,date,time,open,high,low,close,symbol,name,datetime
21225,1,2021-04-08,15:00:00,360.49,364.47,360.29,364.03,AAVE,Aave,2021-04-08 15:00:00
21224,1,2021-04-08,16:00:00,362.06,365.84,362.05,364.79,AAVE,Aave,2021-04-08 16:00:00
21223,1,2021-04-08,17:00:00,366.52,369.81,365.50,368.94,AAVE,Aave,2021-04-08 17:00:00
21222,1,2021-04-08,18:00:00,367.14,368.52,363.89,364.85,AAVE,Aave,2021-04-08 18:00:00
21221,1,2021-04-08,19:00:00,365.81,368.92,365.17,368.14,AAVE,Aave,2021-04-08 19:00:00


### Feature Engineering

In [16]:
# (a) Hourly return
merged['hourly_return'] = merged.groupby('symbol')['close'].pct_change()

# (b) Log return
merged['log_return'] = np.log(merged['close'] / merged.groupby('symbol')['close'].shift(1))

# (c) 3-hour and 6-hour simple moving average of close price
merged['sma_3h'] = merged.groupby('symbol')['close'].transform(lambda x: x.rolling(3).mean())
merged['sma_6h'] = merged.groupby('symbol')['close'].transform(lambda x: x.rolling(6).mean())

# (d) Volatility (standard deviation of close price over a 6-hour window)
merged['volatility_6h'] = merged.groupby('symbol')['close'].transform(lambda x: x.rolling(6).std())

# (e) High-Low spread
merged['hl_spread'] = merged['high'] - merged['low']

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Some Final Cleansing

In [17]:
# Drop columns no longer needed
merged.drop(columns=['date', 'time'], inplace=True)

# Drop rows with NaNs from rolling/pct_change
merged.dropna(inplace=True)

# Reset index and export the cleansed dataset!
merged.reset_index(drop=True, inplace=True)
merged.to_csv("Cleaned_Crypto.csv", index=False)